In [ ]:
import pandas as pd
import nd2
import argparse
import os
import  matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from collections import defaultdict
from datetime import datetime
import shutil

from nis2pyr.reader import read_nd2file
def load_7D(fpath):    
    print(f'Loading from {fpath}')
    with nd2.ND2File(fpath) as nd2file:
        data = read_nd2file(nd2file)
    return data



In [ ]:
data_dir = '/facility/imganfacusers/Elisa/DIF17/DIF_17_1'
fnames = [x for x in sorted(os.listdir(data_dir)) if x.endswith('0001.nd2')]


In [ ]:
output_dir = '/group/jug/ashesh/data/Elisa/patchdataset/puncta'

df_back = pd.read_csv(f'/group/jug/ashesh/data/Elisa/DIF17/background-cropdict.txt')
df_fore = pd.read_csv(f'/group/jug/ashesh/data/Elisa/DIF17/foreground-cropdict.txt')
cur_df = None
if 'foreground' == os.path.basename(output_dir):
    print('Foreground')
    cur_df = df_fore
elif 'puncta' == os.path.basename(output_dir):
    print('Puncta')
    cur_df = df_back
else:
    raise ValueError('Unknown dataset')

## Saving foreground patches

In [ ]:
from disentangle.core.tiff_reader import save_tiff

def get_patch(pd_ser, img):
    z_idx = pd_ser.z_idx
    hs = pd_ser.hs
    he = pd_ser.he
    ws = pd_ser.ws
    we = pd_ser.we
    return img[0,0,z_idx,1,hs:he,ws:we,0]

cur_fpath = None
cur_data = None
for idx in range(cur_df.shape[0]):
    if cur_fpath is None or cur_fpath != cur_df.iloc[idx].fname:
        cur_fpath = cur_df.iloc[idx].fname
        if not os.path.exists(cur_fpath):
            print('Skipping', cur_fpath)
            cur_fpath = None
            continue
        cur_data = load_7D(cur_df.iloc[idx].fname)
    patch = get_patch(cur_df.iloc[idx], cur_data)
    newpath = os.path.join(output_dir, f'{idx}.tiff')
    print(f'Saving to {newpath}')
    save_tiff(newpath, patch)
    # break

In [ ]:
plt.imshow(patch)